In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/csa/CSA3') #切换目录，这个是网盘的根目录
os.getcwd()#获取当前目录

'/content/drive/MyDrive/csa/CSA3'

In [3]:
save_dir = './img'
if os.path.exists(save_dir) is False:
    os.makedirs(save_dir)
save_dir = './img/img8000_feng'
if os.path.exists(save_dir) is False:
    os.makedirs(save_dir)
save_dir = './img/test_mask_big'
if os.path.exists(save_dir) is False:
    os.makedirs(save_dir)
import zipfile
# 原压缩文件在服务器的位置
# 这里的第二个参数用r表示是读取zip文件，w或a是创建一个zip文件
f = zipfile.ZipFile("./img8000.zip",'r')
# f.namelist() 会返回压缩包内所有文件名的列表
for file in f.namelist():
  f.extract(file,"./img/img8000_feng") #解压到的位置，./表示当前目录(与此.ipynb文件同一个目录)
f.close()

f = zipfile.ZipFile("./testing_mask.zip",'r')
# f.namelist() 会返回压缩包内所有文件名的列表
for file in f.namelist():
  f.extract(file,"./img/test_mask_big") #解压到的位置，./表示当前目录(与此.ipynb文件同一个目录)
f.close()

In [1]:
import os
os.getcwd()#获取当前目录

'/content'

In [8]:
class Opion():

    def __init__(self):

        self.dataroot= './testimg/real' #image dataroot
        self.maskroot= './testimg/mask' #mask dataroot

#inpainimg_path = './testimg/real/test.jpg'
#mask_path = './testimg/mask/masktest.png'
        self.batchSize= 1   # Need to be set to 1
        self.fineSize=256 # image size
        self.input_nc=3  # input channel size for first stage
        self.input_nc_g=6 # input channel size for second stage
        self.output_nc=3# output channel size
        self.ngf=64 # inner channel
        self.ndf=64# inner channel
        self.which_model_netD='basic' # patch discriminator

        self.which_model_netF='feature'# feature patch discriminator
        self.which_model_netG='unet_csa'# seconde stage network
        self.which_model_netP='unet_256'# first stage network
        self.triple_weight=1
        self.name='CSA_inpainting'
        self.n_layers_D='3' # network depth
        self.gpu_ids=[0]
        self.model='csa_net'
        self.checkpoints_dir=r'.\checkpoints' #
        self.norm='instance'
        self.fixed_mask=1
        self.use_dropout=False
        self.init_type='normal'
        self.mask_type='center'
        self.lambda_A=100
        self.threshold=5/16.0
        self.stride=1
        self.shift_sz=1 # size of feature patch
        self.mask_thred=1
        self.bottleneck=512
        self.gp_lambda=10.0
        self.ncritic=5
        self.constrain='MSE'
        self.strength=1
        self.init_gain=0.02
        self.cosis=1
        self.gan_type='lsgan'
        self.gan_weight=0.2
        self.overlap=4
        self.skip=0
        self.display_freq=1000
        self.print_freq=50
        self.save_latest_freq=5000
        self.save_epoch_freq=2
        self.continue_train=False
        self.epoch_count=1
        self.phase='train'
        self.which_epoch='30'
        self.niter=20
        self.niter_decay=100
        self.beta1=0.5
        self.lr=0.0002
        self.lr_policy='lambda'
        self.lr_decay_iters=50
        self.isTrain=False



In [15]:
import time
from util.data_load import Data_load
#from models.models import create_model
from models.models import create_model
import torch
import os
import torchvision
from torch.utils import data
import torchvision.transforms as transforms
opt = Opion()
transform_mask = transforms.Compose(
    [transforms.Resize((opt.fineSize,opt.fineSize)),
     transforms.ToTensor(),
    ])
transform = transforms.Compose(
    [
     transforms.Resize((opt.fineSize,opt.fineSize)),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.5] * 3, std=[0.5] * 3)])

dataset_test = Data_load(opt.dataroot, opt.maskroot, transform, transform_mask)
iterator_test = (data.DataLoader(dataset_test, batch_size=opt.batchSize,shuffle=True))
print(len(dataset_test))
model = create_model(opt)
total_steps = 0


1
csa_net


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


initialize network with normal
initialize network with normal
Loading pre-trained network!
model [CSAModel] was created


In [10]:
load_epoch=30
model.load(load_epoch)

AttributeError: 'TestModel' object has no attribute 'load'

In [16]:

save_dir = './measure/true'
if os.path.exists(save_dir) is False:
    os.makedirs(save_dir)

epoch=1
i=0
for image, mask in (iterator_test):
    iter_start_time = time.time()
    image=image.cuda()
    mask=mask.cuda()
    mask=mask[0][0]
    mask=torch.unsqueeze(mask,0)
    mask=torch.unsqueeze(mask,1)
    mask=mask.byte()

    model.set_input(image,mask)
    model.set_gt_latent()
    model.test()
    real_A,real_B,fake_B=model.get_current_visuals()
    pic = (torch.cat([real_A, real_B,fake_B], dim=0) + 1) / 2.0
    torchvision.utils.save_image(pic, '%s/Epoch_(%d)_(%dof%d).jpg' % (
    save_dir, epoch, total_steps + 1, len(dataset_test)), nrow=1)





In [5]:
from PIL import Image, ImageTk
save_dir = './measure/true'
if os.path.exists(save_dir) is False:
    os.makedirs(save_dir)


inpainimg_path = './testimg/real/test.jpg'
inpainimg = Image.open(inpainimg_path)
inpainimg = transform(inpainimg)
input_dict = {
    'A': inpainimg,       # 输入图像的张量
    'A_paths': inpainimg_path   # 输入图像的路径或标识符
}
model.set_input(input_dict)
#model.set_gt_latent()
model.test()
real_A,fake_B=model.get_current_visuals()
pic = (torch.cat([real_A, real_B,fake_B], dim=0) + 1) / 2.0
torchvision.utils.save_image(pic, '%s/Epoch.jpg' % (
    save_dir), nrow=1)

RuntimeError: Given groups=1, weight of size [64, 6, 3, 3], expected input[1, 3, 256, 256] to have 6 channels, but got 3 channels instead

In [18]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 11.7 MB/

In [19]:
import gradio as gr

def greet(name):
    return f"Hello, {name}!"

iface = gr.Interface(fn=greet, inputs="text", outputs="text")
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://7af78e6f1a8a87391d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
